# 导入包及模块

In [64]:
import torch
import torch.nn as nn
from torch.nn import init
import torchvision
import numpy as np
import sys
import torchvision.transforms as transforms

# 获取和读取数据

In [65]:
#获取数据集
mnist_train = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST', train=True, download=True, transform=transforms.ToTensor())
mnist_test = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST', train=False, download=True, transform=transforms.ToTensor())
#读取数据集
batchSize=256
trainIter=torch.utils.data.DataLoader(mnist_train,batch_size=batchSize,shuffle=True,num_workers=8)
testIter=torch.utils.data.DataLoader(mnist_test,batch_size=batchSize,shuffle=True,num_workers=8)

# 定义及初始化模型

定义模型

In [66]:
numInputs = 784
numOutputs = 10

#展平
class FlattenLayer(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self,x):
        return x.view(x.shape[0],-1)
class LinearNet(nn.Module):
    def __init__(self,numInputs,numOutputs):
        super().__init__()
        self.Flatten = FlattenLayer()
        self.linear = nn.Linear(numInputs,numOutputs)
    def forward(self,x):
        x=self.Flatten(x)
        return self.linear(x)
net=LinearNet(numInputs,numOutputs)
print(net) 

LinearNet(
  (Flatten): FlattenLayer()
  (linear): Linear(in_features=784, out_features=10, bias=True)
)


初始化参数

In [67]:
init.normal_(net.linear.weight,mean=0,std=0.01)
init.constant_(net.linear.bias,val=0)

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

# 定义损失函数

In [68]:
loss = nn.CrossEntropyLoss()

# 定义优化算法

In [69]:
optimizer = torch.optim.SGD(net.parameters(),lr=0.1)

# 计算分类准确率

In [70]:
def evaluateAccuary(dataIter,net):
    accSum=0
    n=0
    for x,y in dataIter:
        accSum+=(net(x).argmax(dim=1)==y).float().sum().item()
        n+=y.shape[0]
    return accSum/n

# 训练模型

In [71]:
epochsNum=5

for epoch in range(epochsNum):
    train_l_sum=0
    n=0
    for x,y in trainIter:
        yP=net(x)
        l=loss(yP,y).sum()
        l.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_l_sum+=l
        n+=y.shape[0]
    print(epoch,train_l_sum.item()/n,evaluateAccuary(testIter,net))


0 0.0030784469604492187 0.7909
1 0.0022374160766601564 0.8029
2 0.002058941014607747 0.8133
3 0.001964560953776042 0.8125
4 0.001901702880859375 0.8253
